uvicorn main:app --reload

ETL

In [ ]:
import pandas as pd
import json
import ast
import pyarrow

userItemsData=pd.read_parquet('userItems.parquet')
userItemCountData=pd.read_parquet('userItemCount.parquet')
userItemsExplodedData=pd.read_parquet('userItemsExploded.parquet')
steamGamesData=pd.read_parquet('steamGames.parquet',low_memory=False)
steamGamesPriceData=pd.read_parquet('steamGamesPrice.parquet')
userReviewsExplodedData=pd.read_parquet('userReviewsExploded.parquet')
userItemsDf=pd.DataFrame(userItemsData)
userItemCountDf=pd.DataFrame(userItemCountData)
userItemsExplodedDf=pd.DataFrame(userItemsExplodedData)
steamGamesDf=pd.DataFrame(steamGamesData)
steamGamesPriceDf=pd.DataFrame(steamGamesPriceData)
userReviewsExplodedDf=pd.DataFrame(userReviewsExplodedData)

Steam Games

In [ ]:
with open('output_steam_games.json', 'r') as file:
  lines = file.readlines()

steamGamesData = [json.loads(line) for line in lines]

steamGames = pd.json_normalize(steamGamesData)

steamGamesDf=pd.DataFrame(steamGames)

In [ ]:
steamGamesDf['release_date']=pd.to_datetime(steamGamesDf['release_date'],format='mixed',errors='coerce')
steamGamesDf['release_year'] = steamGamesDf['release_date'].dt.year.fillna(0).astype(int)

In [ ]:
steamGamesDf.rename(columns={'id': 'item_id'}, inplace=True)

In [ ]:
steamGamesDf['price'].loc[steamGamesDf['price'].apply(type) == str].unique()

In [ ]:
steamGamesDf.loc[steamGamesDf['price'].str.contains('third', case=False) & steamGamesDf['price'].notna(), 'price'] = 0

In [ ]:
steamGamesDf.to_parquet('steamGames.parquet',index=False)

In [ ]:
steamGamesPriceDf=steamGamesDf[['item_id','price']]
steamGamesPriceDf.to_parquet('steamGamesPrice.parquet',index=False)


In [ ]:
steamGamesGenresDf=steamGamesDf[['item_id','genres','release_year']]
steamGamesGenresDf=steamGamesGenresDf.dropna(subset='genres')
steamGamesGenresDf['genres'] = steamGamesGenresDf['genres'].apply(lambda lst: [element.lower() for element in lst])
steamGamesGenresDf.to_parquet('steamGamesGenres.parquet',index=False)

In [ ]:
steamGamesDevDf=steamGamesDf[['item_id','developer','release_year']]
steamGamesDevDf=steamGamesDevDf.dropna(subset='developer')

In [ ]:
steamGamesDevDf.to_parquet('steamGamesDev.parquet',index=False)

User Items

In [ ]:
#Ruta del archivo JSON
data_list = []
file_path = 'australian_users_items.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
userItemsDf = pd.DataFrame(data_list)

In [ ]:
userItemCount={'user_id': [], 'items_count': []}
userItemCountDf=pd.DataFrame(userItemCount)
userItemCountDf['user_id']=userItemsDf['user_id']
userItemCountDf['items_count']=userItemsDf['items_count']

In [ ]:
del userItemsDf['steam_id']
del userItemsDf['items_count']
del userItemsDf['user_url']

In [ ]:
userItemsExplodedDf=userItemsDf.explode('items').reset_index(drop=True)

In [ ]:
userItemsExplodedDf['item_id']=userItemsExplodedDf['items'].apply(lambda x: x.get('item_id') if isinstance(x, dict) and 'item_id' in x else None)
userItemsExplodedDf['playtime']=userItemsExplodedDf['items'].apply(lambda x: x.get('playtime_forever') if isinstance(x, dict) and 'playtime_forever' in x else None)

In [ ]:
del userItemsExplodedDf['items']

In [ ]:
userItemsExplodedDf.to_parquet('userItemsExploded.parquet',index=False)
userItemsDf.to_parquet('userItems.parquet',index=False)
userItemCountDf.to_parquet('userItemCount.parquet',index=False)

In [ ]:
userItemsDf

In [ ]:
userItemCountDf

In [ ]:
steamGamesGenresDf

In [178]:
steamGamesData=pd.read_parquet('steamGames.parquet')
userReviewsExplodedData=pd.read_parquet('userReviewsExploded.parquet')
steamGamesDevData=pd.read_parquet('steamGamesDev.parquet')
userRecommendData=pd.read_parquet('userReviewsExploded.parquet')
userItemsData=pd.read_parquet('userItemsExploded.parquet')
userItemCountData=pd.read_parquet('userItemCount.parquet')
steamGamesPriceData=pd.read_parquet('steamGamesPrice.parquet')
steamGamesGenres=pd.read_parquet('steamGamesGenres.parquet')
userItemsExplodedData=pd.read_parquet('userItemsExploded.parquet')

Def Developer

In [169]:
def developer(dev):
  steamGamesDf=pd.DataFrame(steamGamesData)
  filteredDf=steamGamesDf[steamGamesDf['developer']==dev]
  totalApps=filteredDf.groupby('release_year').size().reset_index(name='Cantidad de Items')
  filteredDf=filteredDf.dropna(subset=['price'])
  freeApps=filteredDf[filteredDf['price']==0].groupby('release_year').size().reset_index(name='free_items')
  result = pd.merge(totalApps, freeApps, on='release_year', how='left').fillna(0)
  result.rename(columns={'release_year': 'Año'}, inplace=True)
  result['Contenido Free'] = ((result['free_items'] / result['Cantidad de Items']) * 100).round(2).astype(str) + '%'
  result=result[['Año','Cantidad de Items','Contenido Free']].reset_index(drop=True)
  return result
developer('AMPLITUDE Studios')

Def Userdata

In [171]:
def userData(userId):
  userRecommendDf=pd.DataFrame(userRecommendData)
  userItemsDf=pd.DataFrame(userItemsData)
  userItemCountDf=pd.DataFrame(userItemCountData)
  gamePriceDf=pd.DataFrame(steamGamesPriceData)
  #Filtrar por usuario
  userRecommendDf=userRecommendDf[userRecommendDf['user_id']==userId].reset_index(drop=True)
  userItemsDf=userItemsDf[userItemsDf['user_id']==userId].reset_index(drop=True)
  userItemsDf=pd.merge(userItemsDf,steamGamesPriceDf,how='left',on='item_id')
  userItemCountDf=userItemCountDf[userItemCountDf['user_id']==userId].reset_index(drop=True)
  #Componer DataFrame de respuesta
  resultData={'Usuario': [userId]}
  resultDf=pd.DataFrame(resultData)
  resultDf['Dinero Gastado']=pd.to_numeric(userItemsDf['price'], errors='coerce').sum()
  resultDf['% de recomendación']=((userRecommendDf['recommend'].mean()) * 100).round(2).astype(str) + '%'
  resultDf['Cantidad de Items']=userItemCountDf['items_count']
  return resultDf
userData('76561198043472122')

,Usuario,Dinero Gastado,% de recomendación,Cantidad de Items
0,76561198043472122,534.62,0.0%,88


Def UserForGenre

In [172]:
def userForGenre(genre):
  genre=genre.lower()
  steamGamesGenreDf=pd.DataFrame(steamGamesGenres)
  userItemsExplodedDf=pd.DataFrame(userItemsExplodedData)
  steamGamesGenreDf['genres']=steamGamesGenreDf['genres'].apply(lambda lst: [element.lower() for element in lst])
  steamGamesGenreDf=steamGamesGenreDf[steamGamesGenreDf['genres'].apply(lambda x: genre in x if isinstance(x, list) else False)]
  steamGamesGenreDf=pd.merge(steamGamesGenreDf[['item_id','release_year']],userItemsExplodedDf,how='left',on='item_id')
  steamGamesGenreDf=steamGamesGenreDf.dropna(subset='playtime')
  userId=steamGamesGenreDf.groupby('user_id')['playtime'].sum().reset_index().sort_values(by='playtime',ascending=False).iloc[0,0]
  steamGamesGenreDf=steamGamesGenreDf[steamGamesGenreDf['user_id']==userId].groupby('release_year')['playtime'].sum().reset_index().sort_values(by='release_year',ascending=False)
  steamGamesGenreDf['playtime']=(steamGamesGenreDf['playtime']/60).round(2)
  steamGamesGenreDf.rename(columns={'release_year': 'Año'}, inplace=True)
  steamGamesGenreDf.rename(columns={'playtime': 'Horas jugadas'}, inplace=True)
  resultPlaytime=steamGamesGenreDf.to_dict(orient='records')
  result={'Usuario con más horas jugadas para Género ' + genre:userId,'Horas jugadas':resultPlaytime}
  return result
userForGenre('action')

{'Usuario con más horas jugadas para Género action': 'Sp3ctre',
 'Horas jugadas': [{'Año': 2017, 'Horas jugadas': 722.12},
  {'Año': 2016, 'Horas jugadas': 492.93},
  {'Año': 2015, 'Horas jugadas': 5208.52},
  {'Año': 2014, 'Horas jugadas': 2178.18},
  {'Año': 2013, 'Horas jugadas': 2007.68},
  {'Año': 2012, 'Horas jugadas': 6304.93},
  {'Año': 2011, 'Horas jugadas': 2581.6},
  {'Año': 2010, 'Horas jugadas': 1301.38},
  {'Año': 2009, 'Horas jugadas': 1805.43},
  {'Año': 2008, 'Horas jugadas': 3.73},
  {'Año': 2007, 'Horas jugadas': 1879.73},
  {'Año': 2006, 'Horas jugadas': 14.93},
  {'Año': 2005, 'Horas jugadas': 355.65},
  {'Año': 2004, 'Horas jugadas': 2123.52},
  {'Año': 2003, 'Horas jugadas': 127.88},
  {'Año': 2002, 'Horas jugadas': 3.97},
  {'Año': 2001, 'Horas jugadas': 0.22},
  {'Año': 2000, 'Horas jugadas': 1177.4},
  {'Año': 1999, 'Horas jugadas': 0.73},
  {'Año': 1998, 'Horas jugadas': 0.0},
  {'Año': 1996, 'Horas jugadas': 0.0},
  {'Año': 1995, 'Horas jugadas': 3.62},
  {'

Def BestDeveloperYear

In [185]:
def bestDeveloperYear(year):
  userReviewsExplodedDf=pd.DataFrame(userReviewsExplodedData)
  steamGamesDevDf=pd.DataFrame(steamGamesDevData)
  userReviewsExplodedDf = userReviewsExplodedDf[(userReviewsExplodedDf['recommend'] == True) & 
                               (userReviewsExplodedDf['sentiment_analysis'] == '2') & 
                               (userReviewsExplodedDf['review_year'] == year)]
  userReviewsExplodedDf['review_year']=userReviewsExplodedDf['review_year'].astype(int)
  userReviewsExplodedDf=userReviewsExplodedDf.groupby('item_id')['recommend'].count().reset_index()
  steamGamesDevDf=pd.merge(steamGamesDevDf[['item_id','developer']],userReviewsExplodedDf,how='left',on='item_id').sort_values(by='recommend',ascending=False)
  steamGamesDevDf=steamGamesDevDf.groupby('developer')['recommend'].sum().reset_index().sort_values(by='recommend',ascending=False)
  if steamGamesDevDf.iloc[1,1]!=0:
    first=steamGamesDevDf.iloc[0,0]
    second=steamGamesDevDf.iloc[1,0]
    third=steamGamesDevDf.iloc[2,0]
    result=[{'Puesto 1':first},{'Puesto 2':second},{'Puesto 3':third}]
  else:
    result=[{'Puesto 1':None},{'Puesto 2':None},{'Puesto 3':None}]
  return result
bestDeveloperYear(2012)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Re-Logic'},
 {'Puesto 3': 'Facepunch Studios'}]

Def DeveloperReviewAnalysis

In [184]:
def developerReviewsAnalysis(dev):
  dev=dev.lower()
  steamGamesDevDf=pd.DataFrame(steamGamesDevData)
  userReviewsExplodedDf=pd.DataFrame(userReviewsExplodedData)
  steamGamesDevDf=steamGamesDevDf[steamGamesDevDf['developer'].str.lower()== dev]
  userReviewsExplodedDf=userReviewsExplodedDf.groupby('item_id')['sentiment_analysis'].agg([('positive', lambda x: (x == '2').sum()),
    ('negative', lambda x: (x == '0').sum())]).reset_index()
  steamGamesDevDf=pd.merge(steamGamesDevDf,userReviewsExplodedDf,how='left',on='item_id')
  steamGamesDevDf=steamGamesDevDf.groupby('developer')[['positive','negative']].sum().reset_index()
  positive=steamGamesDevDf.iloc[0,1].astype(int).astype(str)
  negative=steamGamesDevDf.iloc[0,2].astype(int).astype(str)
  result={steamGamesDevDf.iloc[0,0]:['Negative = ' + negative,'Positive = ' + positive]}
  return result
developerReviewsAnalysis('valve')

{'Valve': ['Negative = 1113', 'Positive = 6149']}